<a href="https://colab.research.google.com/github/Junhojuno/keras-tutorial/blob/master/05_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### convolutional base (합성곱 층)의 일부를 재훈련시키자.
- convolutional base + fully connected layer 훈련

In [0]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('./gdrive')

In [0]:
!mv gdrive/My\ Drive/master.zip ./

In [0]:
!unzip master.zip

In [0]:
!cp -r deep-learning-with-python-notebooks-master/datasets ./

In [0]:
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150,150,3))

58892288/58889256 [==============================] - 1s 0us/step


In [0]:
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

### block5부터 tuning해보자
### 차례는 아래와 같다.
- 1. pre-training network위에 새로운 network를 추가(여기선 fully connected layer)
- 2. conv_base를 freezing(동결)한다.(conv_base는 가중치 업데이트 되지 않도록 한다.)
- 3. 새로 추가한 network를 훈련시킨다.
- 4. conv_base의 freezing을 일부만 해제한다.
- 5. 그 일부와 훈련한 새 network를 함께 훈련시킨다.

이전 실습한 내용을 저장해둘걸.... <br>
다시 새로운 network를 훈련시키도록 하자...ㅠ

In [0]:
import shutil, os

original_dataset_dir = "./datasets/cats_and_dogs/train/"

base_dir = "./datasets/cats_and_dogs_small"
os.mkdir(base_dir)

# 경로 및 디렉토리 생성
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

# 훈련용 강아지/고양이 데이터셋 디렉토리 생성
train_dogs_dir = os.path.join(train_dir, 'dogs')
os.mkdir(train_dogs_dir)
train_cats_dir = os.path.join(train_dir, 'cats')
os.mkdir(train_cats_dir)

# 검증용 강아지/고양이 데이터셋 디렉토리 생성
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
os.mkdir(validation_dogs_dir)
validation_cats_dir = os.path.join(validation_dir, 'cats')
os.mkdir(validation_cats_dir)

# 테스트용 강아지/고양이 데이터셋 디렉토리 생성
test_dogs_dir = os.path.join(test_dir, 'dogs')
os.mkdir(test_dogs_dir)
test_cats_dir = os.path.join(test_dir, 'cats')
os.mkdir(test_cats_dir)

In [0]:
fnames = ['dog.{}.jpg'.format(i) for i in range(1000)] # 처음 1000장 강아지 이미지를 train_dogs_dir로 복사
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    cp_src = os.path.join(train_dogs_dir, fname)
    shutil.copyfile(src, cp_src)
    
fnames = ['dog.{}.jpg'.format(i) for i in range(1000,1500)] # 다음 500장 강아지 이미지를 validation_dogs_dir로 복사
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    cp_src = os.path.join(validation_dogs_dir, fname)
    shutil.copyfile(src, cp_src)
    
fnames = ['dog.{}.jpg'.format(i) for i in range(1500,2000)] # 다음 500장 강아지 이미지를 test_dogs_dir로 복사
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    cp_src = os.path.join(test_dogs_dir, fname)
    shutil.copyfile(src, cp_src)
    
fnames = ['cat.{}.jpg'.format(i) for i in range(1000)] # 처음 1000장 고양이 이미지를 train_cats_dir로 복사
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    cp_src = os.path.join(train_cats_dir, fname)
    shutil.copyfile(src, cp_src)
    
fnames = ['cat.{}.jpg'.format(i) for i in range(1000,1500)] # 다음 500장 고양이 이미지를 validation_cats_dir로 복사
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    cp_src = os.path.join(validation_cats_dir, fname)
    shutil.copyfile(src, cp_src)
    
fnames = ['cat.{}.jpg'.format(i) for i in range(1500,2000)] # 다음 500장 고양이 이미지를 test_cats_dir로 복사
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    cp_src = os.path.join(test_cats_dir, fname)
    shutil.copyfile(src, cp_src)

In [0]:
# fully connected layer setting후 위 데이터로 훈련
from keras import models
from keras.layers import *
from keras import optimizers

model = models.Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 16,812,353
Non-trainable params: 0
_________________________________________________________________


In [0]:
# 동결, freezing
# layer당 2개(weight 행렬, bias 벡터)
# VGG가 빠지면서 13개 layer가 빠지고, dense layer 2개만 남음
# 이걸 해주고 compile해줘야함!!

print("conv_base 동결 전 trainable 가중치 수 : ", len(model.trainable_weights))
conv_base.trainable = False
print("conv_base 동결 후 trainable 가중치 수 : ", len(model.trainable_weights))

conv_base 동결 전 trainable 가중치 수 :  30
conv_base 동결 후 trainable 가중치 수 :  4


In [0]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(150,150), batch_size=20, class_mode='binary')
validation_generator = test_datagen.flow_from_directory(validation_dir, target_size=(150,150), batch_size=20, class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [0]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])

hist = model.fit_generator(train_generator, steps_per_epoch=100, epochs=30, validation_data=validation_generator, validation_steps=50, verbose=2)

Epoch 1/30
 - 27s - loss: 0.5869 - acc: 0.7095 - val_loss: 0.4377 - val_acc: 0.8350
Epoch 2/30
 - 25s - loss: 0.4822 - acc: 0.7900 - val_loss: 0.3759 - val_acc: 0.8600
Epoch 3/30
 - 25s - loss: 0.4288 - acc: 0.8145 - val_loss: 0.3312 - val_acc: 0.8650
Epoch 4/30
 - 25s - loss: 0.3972 - acc: 0.8260 - val_loss: 0.3090 - val_acc: 0.8780
Epoch 5/30
 - 25s - loss: 0.3840 - acc: 0.8245 - val_loss: 0.2953 - val_acc: 0.8810
Epoch 6/30
 - 25s - loss: 0.3665 - acc: 0.8410 - val_loss: 0.2963 - val_acc: 0.8810
Epoch 7/30
 - 25s - loss: 0.3596 - acc: 0.8415 - val_loss: 0.2758 - val_acc: 0.8840
Epoch 8/30
 - 25s - loss: 0.3447 - acc: 0.8545 - val_loss: 0.2690 - val_acc: 0.8860
Epoch 9/30
 - 25s - loss: 0.3627 - acc: 0.8430 - val_loss: 0.2656 - val_acc: 0.8870
Epoch 10/30
 - 25s - loss: 0.3414 - acc: 0.8555 - val_loss: 0.2601 - val_acc: 0.8920
Epoch 11/30
 - 25s - loss: 0.3363 - acc: 0.8545 - val_loss: 0.2629 - val_acc: 0.8890
Epoch 12/30
 - 25s - loss: 0.3288 - acc: 0.8650 - val_loss: 0.2578 - val_a

In [0]:
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
  if layer.name == 'block5_conv1':
    set_trainable = True
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False

In [33]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])

hist = model.fit_generator(train_generator, steps_per_epoch=100,epochs=100, validation_data=validation_generator, validation_steps=50)

Epoch 1/100
100/100 [==============================] - 28s 283ms/step - loss: 0.2917 - acc: 0.8715 - val_loss: 0.2287 - val_acc: 0.9050
Epoch 2/100
 68/100 [===================>..........] - ETA: 6s - loss: 0.2816 - acc: 0.8831

KeyboardInterrupt: ignored